# 네이버 지식쇼핑내 순위변경.
#### 가정
 - 자사는 '바디럽'이라고 가정.
 
#### 목표
 - 자사의 상품이 키워드 '마약베개'에 대해서 몇번째 순위에 랭크되어있는지 탐색.

#### 개발 순서
 0. 브라우저 설정
 1. 지식쇼핑 url 분석
 2. 지식쇼핑 '검색 키워드' 조작
 3. HTML 파싱
 4. 자사 상품위치 파악
 
( 5. 필터 적용(네이버랭크순, 낮은 가격순, 높은 가격순, 리뷰 많은수 등))




## 0. 브라우저 설정

In [98]:
# 0. 브라우저 설정
from selenium import webdriver
from pprint import pprint

# 크롬을 사용할겁니다!

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    'download.default_directory': '.',
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
# driver.implicitly_wait(2)

## 1. 지식쇼핑 url 분석

In [99]:
# keyword 설정
# 검색해본다. '오메가3'
# "https://search.shopping.naver.com/search/all.nhn?query=오메가3&frm=NVSHATC" 가 나옵니다.

target_url = "https://search.shopping.naver.com/search/all.nhn?query=오메가3&frm=NVSHATC"
driver.get(target_url)

# 위 코드는 마약베개 검색 결과를 반환합니다.

## 2. 검색 키워드 조작

In [100]:
# 위 코드 처럼 url을 접속하여 HTML을 파싱해도 좋습니다.
# 하지만 코드의 재사용성이 많이 줄어들지요.
# 일일이 검색해 본 후 url을 가져와서 코드를 만드는 건, 효율적이지 않아 보입니다.
# url을 뜯어봅시다.
target_url = "https://search.shopping.naver.com/search/all.nhn?query=오메가3&frm=NVSHATC"
# url은 도메인을 기준으로 여러가지 문자열로 나눌 수 있습니다.
# target_url을 나누어보자면 다음과 같습니다.

# 도메인: https://search.shopping.naver.com/
# url경로: search/all.nhn
# GET parameter: ?query=오메가3&frm=NVSHATC

# url경로의 '?' 뒤에 부분은 GET Parameter로 불립니다. 서버에서 동적인 처리를 하기 위해 받는 '인자(parameter)'입니다.
# 우리는 해당 target_url이 parameter의 query를 기준으로 변경한다는 것을 알 수 있습니다.
# 더 나은 코드는 아래와 같습니다.

base_url = "https://search.shopping.naver.com/search/all.nhn?query="
target_keyword = "오메가3"

target_url = "%s%s" %(base_url, target_keyword)
print(target_url) # 올바른 url이 만들어집니다.
driver.get(target_url)



https://search.shopping.naver.com/search/all.nhn?query=오메가3


In [101]:
# 만약 다른 키워드를 검색하고 싶으시다면 target_keyword만 변경하시면 됩니다.
# 위 코드 복사
base_url = "https://search.shopping.naver.com/search/all.nhn?query="
target_keyword = "마약베개"

target_url = "%s%s" %(base_url, target_keyword)
print(target_url) # 올바른 url이 만들어집니다.
driver.get(target_url)


https://search.shopping.naver.com/search/all.nhn?query=마약베개


## 3. HTML 파싱

In [102]:
# HTML을 분석해야겠지요.
# 그래야 우리가 원하는대로 순서를 매길 수 있잖아요.

# 크롬 "개발자도구>inspect(요소검사)"를 보면 ul.goods_list > li._itemSection 가 하나의 상품을 의미하는 것을 확인 할 수 있습니다.
# 일단 goods_list를 불러와보지요.
goods_list = driver.find_element_by_class_name('goods_list')


# goods_list에서 각 list를 불러와보지요.
goods = goods_list.find_elements_by_class_name('_itemSection')
print("상품개수:", len(goods)) # 몇개를 가져왔나요?




상품개수: 44


In [103]:
# 우리는 위에서 상품별로 분류했습니다.
# 우리는 상품의 세부사항도 분류하고 싶습니다.
# 이를테면, "상품 제목, 상품 설명, 상품 이미지, 공급회사, 가격" 등으로요!

# 필수적으로 세부사항에 대한 html을 분석하여야 합니다.



# image_url: "div.img_area>a.img>img['src']"

# link: "div.info>a['href']"
# price: "div.info>span.price>em.text
# is_ad: ".ad_stk" # (광고인지 확인합니다.)


# review_num: "div.info>span.etc>span[1]"
# register_date: "div.info>span.etc>span[3]"
# jjim_num: "div.info>span.etc>a[1]>em._keepCount"

# mall_name: "div.info_mall>p.mall_txt>a.mall_img".text
# mall_link: "div.info_mall>p.mall_txt>a.mall_img['href']"
# deliver_fee: "div.info_mall>ul.mall_option>li[2]"


# 자 분석이 완료되었습니다.
# 이제 샘플로 하나만 분석해볼까요?
# 'goods'변수에 저장되어있던 첫번째 element로 테스트를 해봅시다.
sample_good = goods[0]
# html문서가 복잡하므로 한번에 xpath로 element를 find합시다.


image_url = sample_good.find_element_by_xpath('div[@class="img_area"]/a[@class="img"]/img').get_attribute('src')

link = sample_good.find_element_by_xpath('div[@class="info"]/a').get_attribute('href')
price = sample_good.find_element_by_xpath('div[@class="info"]/span[@class="price"]/em').text
is_ad = sample_good.find_elements_by_class_name('ad_stk')
if is_ad:
    is_ad = True
else:
    is_ad = False


review_num = sample_good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/span[1]').text
register_date = sample_good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/span[3]').text
jjim_num = sample_good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/a[1]/em').text

mall_name = sample_good.find_element_by_xpath('div[@class="info_mall"]/p[@class="mall_txt"]/a[@class="mall_img"]').text
mall_link = sample_good.find_element_by_xpath('div[@class="info_mall"]/p[@class="mall_txt"]/a[@class="mall_img"]').get_attribute('href')


# 위에 것들을 dictionary 형태로 만들어 봅시다.
good_dict = {
    'is_ad': is_ad,
    'image_url': image_url,
    'link': link,
    'price': price,
    'review_num': review_num,
    'register_date': register_date,
    'jjim_num':jjim_num,
    'mall_name': mall_name,
    'mall_link': mall_link,
}
print(good_dict)


## 잘 가져 왔지요? 샘플에 대한 테스트가 끝났습니다.
# 그러므로 우리는 앞서 가져온 goods list에 반복문을 돌려, 이러한 dictionary의 리스트를 만들겁니다.

{'is_ad': True, 'image_url': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fsearchad-phinf.pstatic.net%2FMjAxODEyMTJfNzkg%2FMDAxNTQ0NTgzNTY4MjE0.yGGKg59owhTnaYZfnDkxRILD132LMucp6zdSsJqRmeIg.U3SvOO4jFwcDcHhBd78RsNimYRAa1CX7q7_567d1lzAg.JPEG%2F1042489-09f86a61-818e-44d6-af3a-d0f5a7b87597.jpg&type=w&size=200', 'link': 'https://adcr.naver.com/adcr?x=KXah5Ug0rxvqqOClyOIdLP///w==k2AcSePZYuOEUM7SLSr050bZ8V6BSjworbsxhvyUYj3Rp4FLFjYL4wdnYDKAhcjhwvliNP24lNOJ1bzZrKCpyAkWXiKKR1fS4vH+GH7tm8++1I6XZ3SZjTDhLIm7eZjAwAMma+3/oTloVExmtbh7iywUKGIbkYP2a0G/+jkllXHastiZ9ML7gPeZNWE3grlpgAlL/X8B33zplgsPbG3KHuTsoA0l5JV2pxgalhhdwr/JNERuTBCM4DHxV45ViFnPbnQtqk9Br+IyK1FqDyd2MBEqLNB7GofY8gTBL0eekUl/JUg6ckSq0viBdddbf7s86zFYTkthiB+Pk3OOL9vG0xrf7Q0t9BlEyr0Cm7fk9/uHxi++tpafoP6OUKQUOwNZ6pUwhnt1OaFTu+ZtZY/rvZJZSrWzA0ZgWK/L4+0j7bgLG9j+FMkBI9Ul5EsQW4SuKwVSXC8KDNHU27S8SQzXUWs1G0/L5N4vjsuKuHDOgEXRjG/qUMJbb7u2b27m0sU538NYcFLwtp/63zLApDXbIvppN2nGadCrFUjZkUGfdlbNxDfuNwNDWzUaN904aUZy8KjBD3PPqge3aOsmmPQxrQ4tkqip90x4CFY+8uivS

In [104]:
# goods list를 dictionary의 list로 만들기

goods_dict_list = []
for good in goods:
    image_url = good.find_element_by_xpath('div[@class="img_area"]/a[@class="img"]/img').get_attribute('src')

    link = good.find_element_by_xpath('div[@class="info"]/a').get_attribute('href')
    price = good.find_element_by_xpath('div[@class="info"]/span[@class="price"]/em').text
    is_ad = good.find_elements_by_class_name('ad_stk')
    print(is_ad)
    if is_ad:
        is_ad = True
    else:
        is_ad = False

    review_num = good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/span[1]').text
    register_date = good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/span[3]').text
    jjim_num = good.find_element_by_xpath('div[@class="info"]/span[@class="etc"]/a[1]/em').text


    mall_name = good.find_element_by_xpath('div[@class="info_mall"]/p[@class="mall_txt"]/a[@class="mall_img"]').text
    mall_link = good.find_element_by_xpath('div[@class="info_mall"]/p[@class="mall_txt"]/a[@class="mall_img"]').get_attribute('href')


    # 위에 것들을 dictionary 형태로 만들어 봅시다.
    good_dict = {
        'is_ad':is_ad,
        'image_url': image_url,
        'link': link,
        'price': price,
        'review_num': review_num,
        'register_date': register_date,
        'jjim_num':jjim_num,
        'mall_name': mall_name,
        'mall_link': mall_link,
    }
    goods_dict_list.append(good_dict)
    
from pprint import pprint
# 성공적으로 저장했습니다..
# 이러면 이쁘게 보여요.
pprint(goods_dict_list)

[<selenium.webdriver.remote.webelement.WebElement (session="da42367f162404ed9ebd559d586692ce", element="0.7901596447620234-49")>]
[<selenium.webdriver.remote.webelement.WebElement (session="da42367f162404ed9ebd559d586692ce", element="0.7901596447620234-57")>]
[<selenium.webdriver.remote.webelement.WebElement (session="da42367f162404ed9ebd559d586692ce", element="0.7901596447620234-65")>]
[<selenium.webdriver.remote.webelement.WebElement (session="da42367f162404ed9ebd559d586692ce", element="0.7901596447620234-73")>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'image_url': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fsearchad-phinf.pstatic.net%2FMjAxODEyMTJfNzkg%2FMDAxNTQ0NTgzNTY4MjE0.yGGKg59owhTnaYZfnDkxRILD132LMucp6zdSsJqRmeIg.U3SvOO4jFwcDcHhBd78RsNimYRAa1CX7q7_567d1lzAg.JPEG%2F1042489-09f86a61-818e-44d6-af3a-d0f5a7b87597.jpg&type=w&size=200',
  'is_ad': True,
  'jjim_num': '1,154',
  'link': 'https:/

## 4. 자사 쇼핑몰 위치 파악.

In [105]:
my_shop_name = '바디럽'
my_shop_rank = -1
for idx, goods_dict in enumerate(goods_dict_list):
    if goods_dict['mall_name'] == my_shop_name:
        my_shop_rank = idx+1
        break

# 리스트 순위에 없으면
if my_shop_rank == -1:
    print('list 순위에 없습니다.... 슬퍼요.')
else:
    print(my_shop_rank, "번째 있습니다!")
    

1 번째 있습니다!


**- 광고를 제거해봅시다**

In [106]:
my_shop_name = '바디럽'
my_shop_rank = -1
for idx, goods_dict in enumerate(goods_dict_list):
    if goods_dict['is_ad']:
        continue
        
    if goods_dict['mall_name'] == my_shop_name:
        my_shop_rank = idx+1
        break

# 리스트 순위에 없으면
if my_shop_rank == -1:
    print('list 순위에 없습니다.... 슬퍼요.')
else:
    print(my_shop_rank, "번째 있습니다!")
    

14 번째 있습니다!
